In [1]:
import pandas as pd
import numpy as np

# Imports of custom moduls
import Multitours as MT
import utils as ut
import CG

# Data

In [2]:
# Read excel file for rows 1 to 9 for demanda data
ncustomers = 8
nvehicles = 4
vehicleCapacity = [60000 for i in range(nvehicles)]
Customers = pd.read_excel('D:/DIEGO/Gent/2nd Year/Thesis/Example2.xlsx', sheet_name='Sheet1', nrows=ncustomers)
Time = pd.read_excel('D:/DIEGO/Gent/2nd Year/Thesis/Example2.xlsx', sheet_name='Sheet1', skiprows=ncustomers+2, usecols="A:C")
Network = Time[["Origin", "Destination"]]
Customers["Tmax"] = Customers["Capacity"] / Customers["Demand"]

### Parameters

In [3]:
# Vehicle cost parameter
psi = [1000 for i in range(nvehicles)]

# Transportation cost
delta = 100

# Delivery cost
# column Phi of df Costumers with indeces 1 to ncustomers
phi = Customers["Phi"].copy()
phi.index = phi.index + 1

# Holding Cost
h = Customers["Holding"].copy()
h.index = h.index + 1

# Demand
d = Customers["Demand"].copy()
d.index = d.index + 1

# Capacity
k = Customers["Capacity"].copy()
k.index = k.index + 1

# Travel time parameter
shape = (ncustomers + 1, ncustomers + 1)
t = np.full(shape, 100)

for index, row in Time.iterrows():
    i = row['Origin']
    j = row['Destination']
    t[i, j] = row['Time']

# Create a dictionary with all the parameters
parameters = {
    "ncustomers": ncustomers,
    "nvehicles": nvehicles,
    "vehicleCapacity": vehicleCapacity,
    "psi": psi,
    "delta": delta,
    "phi": phi,
    "h": h,
    "d": d,
    "k": k,
    "t": t
}


## Initial set of Multitours and Cycle time

### Random and Basic tours

In [4]:
# Random Multitours
# n_MultiToursR = 10
# MultiToursR = MT.generate_random_multi_tour(n_MultiToursR, Network)
# SingleToursR = [MT.identify_tours(multitour) for multitour in MultiToursR]

# # Basic MultiTours
# MultiToursB = MT.generate_basic_tours2(Network, Time, Customers, ncustomers, vehicleCapacity, delta, phi, h, d)
# SingleToursB = [MT.identify_tours(multitour) for multitour in MultiToursB]
# n_MultiToursB = len(MultiToursB)

# # Cycle time and bounds. Update the MultiTours list with only feasible tours (Possibility to mix basic and random tours)
# # Tmin, Tmax, T_EOQ, T, MultiTours = MT.get_cycle_time(Time, Customers, MultiToursR + MultiToursB, SingleToursR + SingleToursB, n_MultiToursR + n_MultiToursB, vehicleCapacity, delta, phi, h, d)
# Tmin, Tmax, T_EOQ, T, MultiTours = MT.get_cycle_time(Time, Customers, MultiToursB,  SingleToursB, n_MultiToursB, vehicleCapacity, delta, phi, h, d)
# n_MultiTours = len(MultiTours) 

# # Binary representation of the MultiTours
# X = MT.Multitours_Binary(MultiTours, ncustomers)

# # Multitour cost
# Cost = MT.Multitour_Cost(n_MultiTours, ncustomers, psi, T, delta, t, X, phi, h, d)

### Big M (Artificial Variable)

In [5]:
# # Artificial variable / multitour
artificial = [0]
for i in range(1, ncustomers + 1):
    artificial.append(i)
artificial.append(0)
artificial

# Multitours
n_MultiTours = 1
MultiTours = [artificial]
# T = [MT.calculate_travel_time(artificial, Time)]
T = [100]
Cost = [1000000] # Big M
X = MT.Multitours_Binary(MultiTours, ncustomers)

# Column Generation

In [6]:
Min_TC, solutionMP, Min_TC_RMP, solutionRMP, X = CG.ColumnGeneration_heuristic(MultiTours,Cost,T,X, parameters)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-29
Iteration 0 :
Min reduced cost -993700.0
Generated MultiTour [0, 1, 0]
Min total cost RMP 1000000.0
Iteration 1 :
Min reduced cost -977583.3333333334
Generated MultiTour [0, 2, 0]
Min total cost RMP 1000000.0
Iteration 2 :
Min reduced cost -966458.3333333334
Generated MultiTour [0, 3, 0]
Min total cost RMP 1000000.0
Iteration 3 :
Min reduced cost -960308.3333333334
Generated MultiTour [0, 4, 0]
Min total cost RMP 1000000.0
Iteration 4 :
Min reduced cost -993850.0
Generated MultiTour [0, 5, 0]
Min total cost RMP 1000000.0
Iteration 5 :
Min reduced cost -988800.0
Generated MultiTour [0, 6, 0]
Min total cost RMP 1000000.0
Iteration 6 :
Min reduced cost -993850.0
Generated MultiTour [0, 7, 0]
Min total cost RMP 1000000.0
Iteration 7 :
Min reduced cost -988800.0
Generated MultiTour [0, 8, 0]
Min total cost RMP 1000000.0
Iteration 8 :
Min reduced cost -748770.0
Generated MultiTour [0, 4, 6, 7, 0, 5, 8,

In [7]:
# For example 2, with 4 vehicles the LP of the RMP is not integer M=1000000
# [(3, 0.5), (8, 1.0), (15, 0.5), (17, 0.5), (20, 1.0), (21, 0.5)]
solutionRMP

[(3, 0.5), (8, 1.0), (15, 0.5), (17, 0.5), (20, 1.0), (21, 0.5)]

In [12]:
# plot results
# ut.plot_results(results, initialization=" (Basic tours)")